In [2]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.pandas as ps
import pandas as pd
import numpy as np


# Pegando o SparkContext existente ou criando um novo
sc = SparkContext.getOrCreate()

spark = SparkSession.builder \
    .appName('credit_score') \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .config("spark.driver.host", "localhost") \
    .getOrCreate()

# ajustando o nível de log do SparkSession e SparkContext para informar apenas o que estiver a nivel de ERROR
# (necessário em razão das configurações do Apache Spark no meu computador)
sc.setLogLevel("ERROR")
spark.sparkContext.setLogLevel("ERROR")

In [3]:
installments_p = spark.read.csv('datasets/installments_payments.csv', inferSchema=True, header=True)
# bureau_b = spark.read.csv('datasets/bureau_balance.csv', inferSchema=True, header=True)
# bur = spark.read.csv('datasets/bureau.csv', inferSchema=True, header=True)
# merged_app = spark.read.csv('datasets/merged_application.csv', inferSchema=True, header=True)

merged_b = spark.read.csv('datasets/merged_bureau.csv', inferSchema=True, header=True)
# POS_CASH_b = spark.read.csv('datasets/POS_CASH_balance.csv', inferSchema=True, header=True)
# prev_app = spark.read.csv('datasets/previous_application.csv', inferSchema=True, header=True)


In [4]:
cc_balance = pd.read_csv('datasets/credit_card_balance.csv', encoding='ISO-8859-1')
prev_app = pd.read_csv('datasets/previous_application.csv', encoding='ISO-8859-1')
POS_CASH_b = pd.read_csv('datasets/POS_CASH_balance.csv', encoding='ISO-8859-1')
bureau_b = pd.read_csv('datasets/bureau_balance.csv', encoding='ISO-8859-1')
app_train = pd.read_csv('datasets/application_train.csv', encoding='ISO-8859-1')
merged_app = pd.read_csv('datasets/merged_application.csv', encoding='ISO-8859-1')
bur = pd.read_csv('datasets/bureau.csv', encoding='ISO-8859-1')
sample_subm = pd.read_csv('datasets/sample_submission.csv', encoding='ISO-8859-1')
home_cred = pd.read_csv('datasets/HomeCredit_columns_description.csv', encoding='ISO-8859-1')

In [5]:
application_train = app_train
bureau_balance = bureau_b
bureau = bur
credit_card_balance = cc_balance
home_cred_col_descrip = home_cred
installments_payments = installments_p.pandas_api()
merged_application = merged_app
merged_bureau = merged_b.pandas_api()
POS_CASH_balance = POS_CASH_b
previous_app = prev_app
sample_submission = sample_subm

# 1. Conhecendo os datasets

In [6]:
pd.set_option('display.max_colwidth', None)
home_cred_col_descrip[home_cred_col_descrip['Table'] == 'installments_payments.csv']

,Unnamed: 0,Table,Row,Description,Special
211,214,installments_payments.csv,SK_ID_PREV,"ID of previous credit in Home credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loans in Home Credit)",hashed
212,215,installments_payments.csv,SK_ID_CURR,ID of loan in our sample,hashed
213,216,installments_payments.csv,NUM_INSTALMENT_VERSION,Version of installment calendar (0 is for credit card) of previous credit. Change of installment version from month to month signifies that some parameter of payment calendar has changed,NaN
214,217,installments_payments.csv,NUM_INSTALMENT_NUMBER,On which installment we observe payment,NaN
215,218,installments_payments.csv,DAYS_INSTALMENT,When the installment of previous credit was supposed to be paid (relative to application date of current loan),time only relative to the application
216,219,installments_payments.csv,DAYS_ENTRY_PAYMENT,When was the installments of previous credit paid actually (relative to application date of current loan),time only relative to the application
217,220,installments_payments.csv,AMT_INSTALMENT,What was the prescribed installment amount of previous credit on this installment,NaN
218,221,installments_payments.csv,AMT_PAYMENT,What the client actually paid on previous credit on this installment,NaN


## Detalhamento dos Campos

SK_ID_PREV: Cada crédito anterior é identificado por um código único. Este campo relaciona o empréstimo atual com possíveis empréstimos anteriores que o cliente teve na mesma instituição financeira.

SK_ID_CURR: Identificador único para o empréstimo que está sendo analisado na amostra de dados atual.

NUM_INSTALMENT_VERSION: Se houver alterações no calendário de parcelas, como ajustes no prazo ou nas datas, essa coluna ajuda a identificar a versão do calendário para cada parcela.

NUM_INSTALMENT_NUMBER: Indica qual parcela específica do crédito anterior está sendo observada, por exemplo, a primeira, segunda ou terceira parcela.

DAYS_INSTALMENT: Mostra a data programada para o pagamento da parcela em relação à data de aplicação do empréstimo atual. Por exemplo, -30 pode significar que o pagamento era esperado 30 dias antes da data de aplicação do crédito atual.

DAYS_ENTRY_PAYMENT: Mostra a data real do pagamento da parcela em relação à data de aplicação do crédito atual. Se o valor for negativo, indica que o pagamento foi feito antes da data de aplicação.

AMT_INSTALMENT: O valor da parcela conforme o contrato do crédito anterior. Esse valor pode ser alterado ao longo do tempo se houver mudanças nas condições do crédito. Se o contrato diz que a parcela mensal é R 500, esse valor é o AMT_INSTALMENT.

AMT_PAYMENT: O valor efetivamente pago pelo cliente para essa parcela do crédito anterior. Pode ser menor ou maior que o valor da parcela se houver pagamentos antecipados ou atrasos. Se o cliente pagou R 450, em vez dos R 500, esse valor é o AMT_PAYMENT.

In [7]:
# installments_payments.repartition(100)
installments_payments.describe()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
count,1.360540e+07,1.360540e+07,1.360540e+07,1.360540e+07,1.360540e+07,1.360250e+07,1.360540e+07,1.360250e+07
mean,1.903365e+06,2.784449e+05,8.566373e-01,1.887090e+01,-1.042270e+03,-1.051114e+03,1.705091e+04,1.723822e+04
std,5.362029e+05,1.027183e+05,1.035216e+00,2.666407e+01,8.009463e+02,8.005859e+02,5.057025e+04,5.473578e+04
min,1.000001e+06,1.000010e+05,0.000000e+00,1.000000e+00,-2.922000e+03,-4.921000e+03,0.000000e+00,0.000000e+00
25%,1.434140e+06,1.896230e+05,0.000000e+00,4.000000e+00,-1.653000e+03,-1.662000e+03,4.225860e+03,3.397320e+03
50%,1.896486e+06,2.786720e+05,1.000000e+00,8.000000e+00,-8.180000e+02,-8.270000e+02,8.884080e+03,8.124975e+03
75%,2.369086e+06,3.675450e+05,1.000000e+00,1.900000e+01,-3.610000e+02,-3.700000e+02,1.671210e+04,1.610635e+04
max,2.843499e+06,4.562550e+05,1.780000e+02,2.770000e+02,-1.000000e+00,-1.000000e+00,3.771488e+06,3.771488e+06


In [6]:
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [8]:
installments_payments.isna().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64

In [9]:
install_p = installments_payments.dropna()
install_p.isna().sum()

SK_ID_PREV                0
SK_ID_CURR                0
NUM_INSTALMENT_VERSION    0
NUM_INSTALMENT_NUMBER     0
DAYS_INSTALMENT           0
DAYS_ENTRY_PAYMENT        0
AMT_INSTALMENT            0
AMT_PAYMENT               0
dtype: int64

24/07/11 23:46:04 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/11 23:59:27 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 00:39:31 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 01:01:28 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

...

24/07/12 07:17:58 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 07:24:19 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 07:27:56 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 07:50:13 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 07:53:34 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

## 1.1 Coluna `application_train`

In [18]:
pd.set_option('display.max_rows', None)
home_cred_col_descrip[home_cred_col_descrip['Table'] == 'application_{train|test}.csv']
# home_cred_col_descrip['Table'].unique()

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,"Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)",NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
5,8,application_{train|test}.csv,FLAG_OWN_REALTY,Flag if client owns a house or flat,NaN
6,9,application_{train|test}.csv,CNT_CHILDREN,Number of children the client has,NaN
7,10,application_{train|test}.csv,AMT_INCOME_TOTAL,Income of the client,NaN
8,11,application_{train|test}.csv,AMT_CREDIT,Credit amount of the loan,NaN
9,12,application_{train|test}.csv,AMT_ANNUITY,Loan annuity,NaN


24/07/12 18:25:07 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 18:26:54 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 18:34:39 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 18:34:59 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

...

24/07/12 18:42:09 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 18:42:29 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 18:42:49 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 18:43:19 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

24/07/12 18:43:49 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

In [12]:
application_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
application_train.dtypes

SK_ID_CURR                        int64
TARGET                            int64
NAME_CONTRACT_TYPE               object
CODE_GENDER                      object
FLAG_OWN_CAR                     object
FLAG_OWN_REALTY                  object
CNT_CHILDREN                      int64
AMT_INCOME_TOTAL                float64
AMT_CREDIT                      float64
AMT_ANNUITY                     float64
AMT_GOODS_PRICE                 float64
NAME_TYPE_SUITE                  object
NAME_INCOME_TYPE                 object
NAME_EDUCATION_TYPE              object
NAME_FAMILY_STATUS               object
NAME_HOUSING_TYPE                object
REGION_POPULATION_RELATIVE      float64
DAYS_BIRTH                        int64
DAYS_EMPLOYED                     int64
DAYS_REGISTRATION               float64
DAYS_ID_PUBLISH                   int64
OWN_CAR_AGE                     float64
FLAG_MOBIL                        int64
FLAG_EMP_PHONE                    int64
FLAG_WORK_PHONE                   int64


In [30]:
import scipy.stats as ss
from sklearn.preprocessing import LabelEncoder

# Função para calcular Cramér's V
def cramers_v(confusion_matrix):
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()  # Soma total de todos os elementos da matriz de contingência
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1)) / (n-1))
    rcorr = r - ((r-1)**2) / (n-1)
    kcorr = k - ((k-1)**2) / (n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))


# Lista para armazenar os resultados
results = []

# Iterar sobre colunas categóricas
for column in application_train.select_dtypes(include=['object']).columns:
    confusion_matrix = pd.crosstab(application_train[column], application_train['TARGET'])
    chi2, p, dof, ex = ss.chi2_contingency(confusion_matrix)
    cramers_v_value = cramers_v(confusion_matrix)
    results.append((column, chi2, p, cramers_v_value))

# Criar DataFrame com os resultados
results_df = pd.DataFrame(results, columns=['Variable', 'Chi2', 'p-value', 'Cramers V'])
results_df = results_df.sort_values(by='Cramers V', ascending=False)

# Mostrar os resultados
print(results_df)


                      Variable         Chi2        p-value  Cramers V
9              OCCUPATION_TYPE  1402.846796  3.784500e-288   0.081020
11           ORGANIZATION_TYPE  1609.240636  5.224541e-299   0.071048
5             NAME_INCOME_TYPE  1253.470808  1.928146e-266   0.063667
6          NAME_EDUCATION_TYPE  1019.213187  2.447681e-219   0.057458
1                  CODE_GENDER   920.791334  1.129022e-200   0.054661
7           NAME_FAMILY_STATUS   504.694083  7.744842e-107   0.040311
8            NAME_HOUSING_TYPE   420.556190   1.099089e-88   0.036761
0           NAME_CONTRACT_TYPE   293.150542   1.023515e-65   0.030823
14          WALLSMATERIAL_MODE   139.235314   1.453180e-27   0.029688
2                 FLAG_OWN_CAR   146.656018   9.330994e-34   0.021764
13              HOUSETYPE_MODE    27.632556   9.992328e-07   0.012934
12          FONDKAPREMONT_MODE    16.809897   7.732982e-04   0.011919
15         EMERGENCYSTATE_MODE    23.678150   1.138680e-06   0.011841
4              NAME_

In [32]:
confusion_matrix

TARGET,0,1
EMERGENCYSTATE_MODE,,
No,148324,11104
Yes,2105,223


In [ ]:
SK_ID_CURR: Identificação do empréstimo na amostra. Cada linha corresponde a um empréstimo diferente.

TARGET: 1 - cliente teve dificuldades de pagamento (atrasos em pelo menos uma das primeiras parcelas do empréstimo)
        0 - cliente não teve dificuldades

NAME_CONTRACT_TYPE: Tipo de contrato do empréstimo. Pode ser Cash loans (empréstimo em dinheiro) ou Revolving loans (empréstimo rotativo, como cartão de crédito).

CODE_GENDER: Gênero do cliente (M para masculino, F para feminino).

FLAG_OWN_CAR: Indicador se o cliente possui um carro (Y para sim, N para não).
    
AMT_REQ_CREDIT_BUREAU_DAY: Número de consultas ao bureau de crédito sobre o cliente um dia antes da aplicação (excluindo consultas uma hora antes da aplicação).

AMT_REQ_CREDIT_BUREAU_WEEK: Número de consultas ao bureau de crédito sobre o cliente uma semana antes da aplicação (excluindo consultas um dia antes da aplicação).

AMT_REQ_CREDIT_BUREAU_MON: Número de consultas ao bureau de crédito sobre o cliente um mês antes da aplicação (excluindo consultas uma semana antes da aplicação).

AMT_REQ_CREDIT_BUREAU_QRT: Número de consultas ao bureau de crédito sobre o cliente três meses antes da aplicação (excluindo consultas um mês antes da aplicação).

AMT_REQ_CREDIT_BUREAU_YEAR: Número de consultas ao bureau de crédito sobre o cliente um ano antes da aplicação (excluindo consultas nos últimos três meses antes da aplicação).



In [8]:
application_train.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


In [35]:
display(bureau_balance)

SK_ID_BUREAU,MONTHS_BALANCE,STATUS
5715448,0,C
5715448,-1,C
5715448,-2,C
5715448,-3,C
5715448,-4,C
5715448,-5,C
5715448,-6,C
5715448,-7,C
5715448,-8,C
5715448,-9,0


In [36]:
display(bureau)

SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NULL,0,91323.0,0.0,NULL,0.0,Consumer credit,-131,NULL
215354,5714463,Active,currency 1,-208,0,1075.0,NULL,NULL,0,225000.0,171342.0,NULL,0.0,Credit card,-20,NULL
215354,5714464,Active,currency 1,-203,0,528.0,NULL,NULL,0,464323.5,NULL,NULL,0.0,Consumer credit,-16,NULL
215354,5714465,Active,currency 1,-203,0,NULL,NULL,NULL,0,90000.0,NULL,NULL,0.0,Credit card,-16,NULL
215354,5714466,Active,currency 1,-629,0,1197.0,NULL,77674.5,0,2700000.0,NULL,NULL,0.0,Consumer credit,-21,NULL
215354,5714467,Active,currency 1,-273,0,27460.0,NULL,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,NULL
215354,5714468,Active,currency 1,-43,0,79.0,NULL,0.0,0,42103.8,42103.8,0.0,0.0,Consumer credit,-22,NULL
162297,5714469,Closed,currency 1,-1896,0,-1684.0,-1710.0,14985.0,0,76878.45,0.0,0.0,0.0,Consumer credit,-1710,NULL
162297,5714470,Closed,currency 1,-1146,0,-811.0,-840.0,0.0,0,103007.7,0.0,0.0,0.0,Consumer credit,-840,NULL
162297,5714471,Active,currency 1,-1146,0,-484.0,NULL,0.0,0,4500.0,0.0,0.0,0.0,Credit card,-690,NULL


In [37]:
display(credit_card_balance)

SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
2562384,378907,-6,56.97,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.0,0.0,0.0,0.0,1,0.0,1.0,35.0,Active,0,0
2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.0,2250.0,2250.0,60175.08,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.0,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
1389973,337855,-4,236572.11,225000,2250.0,2250.0,0.0,0.0,11795.76,11925.0,11925.0,224949.285,233048.97,233048.97,1.0,1,0.0,0.0,10.0,Active,0,0
1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.89,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
2646502,380010,-7,82903.815,270000,0.0,0.0,0.0,0.0,4449.105,3825.0,3825.0,80519.04,82773.315,82773.315,0.0,0,0.0,0.0,2.0,Active,7,0
1079071,171320,-6,353451.645,585000,67500.0,67500.0,0.0,0.0,14684.175,15750.0,15750.0,345433.86,351881.145,351881.145,1.0,1,0.0,0.0,6.0,Active,0,0
2095912,118650,-7,47962.125,45000,45000.0,45000.0,0.0,0.0,0.0,264.69,0.0,44735.31,47962.125,47962.125,1.0,1,0.0,0.0,51.0,Active,0,0
2181852,367360,-4,291543.075,292500,90000.0,289339.425,0.0,199339.425,130.5,4093.515,4093.515,285376.41,286831.575,286831.575,3.0,8,0.0,5.0,3.0,Active,0,0
1235299,203885,-5,201261.195,225000,76500.0,111026.7,0.0,34526.7,6338.34,45000.0,45000.0,192793.275,197224.695,197224.695,3.0,9,0.0,6.0,38.0,Active,0,0


In [13]:
pd.set_option('display.max_colwidth', None)
home_cred_col_descrip.head(20)

,Unnamed: 0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,"Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)",NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
5,8,application_{train|test}.csv,FLAG_OWN_REALTY,Flag if client owns a house or flat,NaN
6,9,application_{train|test}.csv,CNT_CHILDREN,Number of children the client has,NaN
7,10,application_{train|test}.csv,AMT_INCOME_TOTAL,Income of the client,NaN
8,11,application_{train|test}.csv,AMT_CREDIT,Credit amount of the loan,NaN
9,12,application_{train|test}.csv,AMT_ANNUITY,Loan annuity,NaN


In [46]:
pandas_df = home_cred_col_descrip.toPandas()
pandas_df.shape

(219, 5)

In [14]:
# selecionar apenas as colunas que interessam para aprender sobre elas

In [ ]:
#MODELO

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

# Carregar um conjunto de dados de exemplo
data = load_iris()
X, y = data.data, data.target

# Introduzir alguns valores ausentes para demonstrar a imputação
X[0, 0] = np.nan
X[2, 2] = np.nan

# Criar um pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Passo 1: Imputação de valores ausentes
    ('scaler', StandardScaler()),                # Passo 2: Escalonamento dos dados
    ('classifier', RandomForestClassifier())     # Classificador de exemplo
])

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar o pipeline aos dados de treino
pipeline.fit(X_train, y_train)

# Avaliar o pipeline nos dados de teste
accuracy = pipeline.score(X_test, y_test)
print("Acurácia:", accuracy)

In [ ]:
# TRATANDO DIFERENTES COLUNAS DE FORMAS DIFERENTES

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Criar um DataFrame de exemplo
data = {
    'age': [25, np.nan, 35, 45, 22],
    'salary': [50000, 60000, np.nan, 100000, 30000],
    'city': ['New York', 'San Francisco', 'New York', np.nan, 'Chicago'],
    'bought': [1, 0, 1, 0, 1]
}
df = pd.DataFrame(data)

# Separar características e alvo
X = df.drop('bought', axis=1)
y = df['bought']

# Definir transformações para diferentes tipos de colunas
numeric_features = ['age', 'salary']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_features = ['city']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar transformações usando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Criar um pipeline que inclui o preprocessador e o modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar o pipeline aos dados de treino
pipeline.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred = pipeline.predict(X_test)

# Avaliar o pipeline nos dados de teste
accuracy = pipeline.score(X_test, y_test)
print("Acurácia:", accuracy)

Exemplo de Ordem para um Projeto de Machine Learning:
Desenvolva um pipeline:

Defina as etapas de pré-processamento e modelagem.
Crie e teste um pipeline básico.
Detecte e trate a multicolinearidade:

Inspecione as variáveis no pipeline.
Aplique técnicas para ajustar a multicolinearidade, se necessário.
Realize a cross-validation:

Aplique a cross-validation usando o pipeline finalizado para avaliar o desempenho do modelo.
Resumindo
Pipeline: Crie e estruture seu fluxo de trabalho.
Detecção de Multicolinearidade: Verifique e ajuste variáveis.
Cross-Validation: Avalie o desempenho do modelo.

In [ ]:
# PIPELINE PARA TRATAR AS COLUNAS TAMBÉM!!!

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Definindo o transformador personalizado
class EmpLengthTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_transformed = X.copy()
        X_transformed = X_transformed.str.replace('+ years', '')
        X_transformed = X_transformed.str.replace('< 1 year', '0')
        X_transformed = X_transformed.str.replace(' years', '')
        X_transformed = X_transformed.str.replace(' year', '')
        X_transformed = X_transformed.fillna('0')
        return X_transformed

# Exemplo de dados
data = {
    'emp_length': ['10+ years', '2 years', '< 1 year', '3 years', '5 years', None],
    'feature1': [5, 3, 1, 4, 2, 6],
    'target': [1, 0, 1, 0, 1, 0]
}
datos = pd.DataFrame(data)

# Separando características e alvo
X = datos[['emp_length', 'feature1']]
y = datos['target']

# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definindo o ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('emp_length', EmpLengthTransformer(), 'emp_length')
    ],
    remainder='passthrough'  # mantém as outras colunas inalteradas
)

# Definindo o pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Ajustando o pipeline aos dados de treinamento
pipeline.fit(X_train, y_train)

# Transformando os dados de teste e fazendo previsões
y_pred = pipeline.predict(X_test)

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

In [ ]:
# WEIGHT OF EVIDENCE

import pandas as pd
import numpy as np

# Função para calcular WoE
def calculate_woe(df, var, target):
    eps = 1e-10  # Small value to avoid division by zero
    df = df[[var, target]].copy()
    df['good'] = (df[target] == 0).astype(int)
    df['bad'] = (df[target] == 1).astype(int)
    
    grouped = df.groupby(var).agg({'good': 'sum', 'bad': 'sum'})
    grouped['good_pct'] = grouped['good'] / grouped['good'].sum() + eps
    grouped['bad_pct'] = grouped['bad'] / grouped['bad'].sum() + eps
    grouped['woe'] = np.log(grouped['good_pct'] / grouped['bad_pct'])
    
    return grouped[['woe']].reset_index()

# Função para aplicar WoE às variáveis
def apply_woe(df, var, woe_df):
    return df.merge(woe_df, on=var, how='left').drop(columns=[var])

# Exemplo de uso
data = pd.DataFrame({
    'age': [25, 45, 35, 50, 23, 43, 36],
    'income': [50000, 100000, 75000, 120000, 60000, 95000, 80000],
    'default': [0, 1, 0, 1, 0, 1, 0]
})

# Binning da variável 'age'
data['age_bin'] = pd.cut(data['age'], bins=[20, 30, 40, 50, 60], labels=['20-30', '30-40', '40-50', '50-60'])

# Calcular WoE para 'age_bin'
woe_age = calculate_woe(data, 'age_bin', 'default')

# Aplicar WoE
data_woe = apply_woe(data, 'age_bin', woe_age)
print(data_woe)

In [ ]:
# FEATURE IMPORTANCE + WOE

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Suponha que você tenha uma lista de DataFrames
dataframes = [df1, df2, df3, ...]  # Substitua com seus DataFrames reais

# Passo 1: Calcular Feature Importance
def calculate_feature_importance(df, target_column):
    X = df.drop(columns=[target_column])
    y = df[target_column]
    model = RandomForestClassifier()
    model.fit(X, y)
    importances = model.feature_importances_
    feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
    return feature_importance.sort_values(by='Importance', ascending=False)

# Passo 2: Selecionar as variáveis mais importantes
important_features = []
for df in dataframes:
    feature_importance = calculate_feature_importance(df, 'default')
    important_features.extend(feature_importance.head(10)['Feature'].tolist())  # Selecionando as 10 mais importantes de cada DataFrame

important_features = list(set(important_features))  # Remover duplicatas

# Passo 3: Calcular WoE para as variáveis selecionadas
def calculate_woe(df, var, target):
    eps = 1e-10
    df = df[[var, target]].copy()
    df['good'] = (df[target] == 0).astype(int)
    df['bad'] = (df[target] == 1).astype(int)
    
    grouped = df.groupby(var).agg({'good': 'sum', 'bad': 'sum'})
    grouped['good_pct'] = grouped['good'] / grouped['good'].sum() + eps
    grouped['bad_pct'] = grouped['bad'] / grouped['bad'].sum() + eps
    grouped['woe'] = np.log(grouped['good_pct'] / grouped['bad_pct'])
    
    return grouped[['woe']].reset_index()

def apply_woe(df, var, woe_df):
    return df.merge(woe_df, on=var, how='left').drop(columns=[var])

for df in dataframes:
    for feature in important_features:
        if feature in df.columns:
            if df[feature].dtype == 'object':
                woe_df = calculate_woe(df, feature, 'default')
                df = apply_woe(df, feature, woe_df)
            else:
                # Binning para variáveis numéricas
                df[f'{feature}_bin'] = pd.qcut(df[feature], q=10, duplicates='drop')
                woe_df = calculate_woe(df, f'{feature}_bin', 'default')
                df = apply_woe(df, f'{feature}_bin', woe_df)

# Passo 4: Treinar o modelo final com variáveis transformadas
X = df[important_features]
y = df['default']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)
accuracy = accuracy_score(y_test, preds)
print(f'Accuracy: {accuracy}')